In [19]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve, precision_score,recall_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt

In [22]:
product_train= pd.read_csv("product_train.csv")
product_test=pd.read_csv("product_test.csv")

In [ ]:

product_train.head()


,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,3921548,8,12,0,0,0,0,1,1,2,...,0,0.63,0.75,0,No,No,No,Yes,No,No
1,3191009,83,2,33,157,377,603,44,98,148,...,0,0.68,0.66,0,No,No,No,Yes,No,No
2,2935810,8,4,0,0,0,0,0,0,1,...,0,0.73,0.78,0,No,No,No,Yes,No,No
3,2205847,31,4,63,70,160,223,27,90,164,...,0,0.73,0.78,0,No,No,Yes,Yes,No,No
4,4953497,3,12,0,0,0,0,0,0,0,...,0,0.81,0.74,0,No,No,No,Yes,No,No


In [ ]:
product_test.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,potential_issue,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop
0,3799238,2,8,0,0,0,0,0,0,1,...,No,0,0.98,0.99,0,No,No,No,Yes,No
1,3268515,149,8,42,238,438,598,56,192,399,...,No,0,0.84,0.87,0,No,No,No,Yes,No
2,2857331,11,8,0,0,0,0,0,0,0,...,No,0,0.40,0.42,0,No,No,No,Yes,No
3,3320909,272,4,0,0,0,0,2,4,5,...,No,0,0.73,0.78,0,No,No,No,Yes,No
4,3128606,41,8,0,0,0,0,0,0,0,...,No,0,0.99,0.99,0,No,No,No,Yes,No


In [ ]:
product_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250078 entries, 0 to 250077
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   sku                250078 non-null  int64  
 1   national_inv       250078 non-null  int64  
 2   lead_time          250078 non-null  int64  
 3   in_transit_qty     250078 non-null  int64  
 4   forecast_3_month   250078 non-null  int64  
 5   forecast_6_month   250078 non-null  int64  
 6   forecast_9_month   250078 non-null  int64  
 7   sales_1_month      250078 non-null  int64  
 8   sales_3_month      250078 non-null  int64  
 9   sales_6_month      250078 non-null  int64  
 10  sales_9_month      250078 non-null  int64  
 11  min_bank           250078 non-null  int64  
 12  potential_issue    250078 non-null  object 
 13  pieces_past_due    250078 non-null  int64  
 14  perf_6_month_avg   250078 non-null  float64
 15  perf_12_month_avg  250078 non-null  float64
 16  lo

In [23]:
def change(row):
    if row['went_on_backorder'] == 'No':
        val = 0
    elif row['went_on_backorder'] == 'Yes':
        val = 1
    else:
        val = None
    return val

product_train['went_on_backorder'] = product_train.apply(change, axis=1)

categorical_columns=['deck_risk','ppap_risk','stop_auto_buy','oe_constraint','rev_stop','potential_issue']
for i in categorical_columns:

    def change(row):
        if row[i]=='No':
            val=0
        elif row[i]=='Yes':
            val=1
        else:
            val=None
        return val
    product_train[i]=product_train.apply(change,axis=1)
    product_test[i]=product_test.apply(change,axis=1)

In [24]:
product_train['went_on_backorder'].value_counts()

,count
went_on_backorder,
0,248463
1,1615


In [25]:
for i in categorical_columns:
    print(product_train[i].value_counts())

deck_risk
0    206295
1     43783
Name: count, dtype: int64
ppap_risk
0    221402
1     28676
Name: count, dtype: int64
stop_auto_buy
1    243586
0      6492
Name: count, dtype: int64
oe_constraint
0    250036
1        42
Name: count, dtype: int64
rev_stop
0    250018
1        60
Name: count, dtype: int64
potential_issue
0    249944
1       134
Name: count, dtype: int64


In [26]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Example code for using SMOTE
X = product_train.drop(['went_on_backorder','rev_stop','potential_issue','oe_constraint'], axis=1)  # Features
y = product_train['went_on_backorder']  # Target

# Create SMOTE object
smote = SMOTE(sampling_strategy='minority')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify =y,random_state=10)
scale_pos_weight = len(y_train) / (2 * np.bincount(y_train)[1])

# Apply SMOTE to balance the dataset
from imblearn.combine import SMOTETomek


X_train, y_train = smote.fit_resample(X_train, y_train)


In [27]:
 y_train.value_counts()

,count
went_on_backorder,
0,198770
1,198770


In [ ]:
for i in X_train.columns:
    print(X_train[i].value_counts())

sku
3564293    67
3449957    59
2192250    58
2140165    51
2254049    42
           ..
3419950     1
4800420     1
3715435     1
3747513     1
3036554     1
Name: count, Length: 340470, dtype: int64
national_inv
 0          73385
 1          31660
 2          31085
 3          22192
 4          17549
            ...  
 1024087        1
 5133           1
 6576           1
 3306           1
-855            1
Name: count, Length: 5870, dtype: int64
lead_time
8     136536
2      72960
4      31788
12     28563
9      24100
3      16566
6      15111
5      14721
7      14667
10      9436
11      7733
52      3381
0       3336
14      1823
1       1803
16      1268
13      1171
15       967
17       468
20        72
30        55
24        42
18        39
22        29
40        23
42        19
47        19
26        18
29        17
28        17
45        16
48        16
35        16
23        15
51        15
37        14
50        14
46        13
36        13
43        13
25        13
19    

In [28]:
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier

param_xgb = {'n_estimators': [10,20,50,100,200,500],
             'learning_rate': [0.001,0.01,0.05, 0.1,0.5,1],
             'max_depth': [2,3,4,5,10,20,30,None],
             'min_child_weight': [5,25,50,100,500,1000],
             'gamma': [0,0.01,0.1,0.5,1,2,5],
             'subsample': [0.6,0.8,1.0],
             'colsample_bytree': [0.6,0.8,1.0],
             'objective': ['binary:logistic'],
             'nthread': [-1],
             'scale_pos_weight': [1,3,5,7,9]
             }

from sklearn.model_selection import RandomizedSearchCV
xgb_base = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Initialize AdaBoost classifier with XGBoost as the base estimator
ada_boost = AdaBoostClassifier(estimator=xgb_base, n_estimators=5, random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    ada_boost,
    param_distributions={
        'estimator__' + key: value for key, value in param_xgb.items()
    },
    n_iter=5,
    scoring='accuracy',  # or use 'f1', 'precision', 'recall' as per your requirement
    cv=5,
    n_jobs=-1,
    random_state=42
)

#xgb_fit = RandomizedSearchCV(xgb.XGBClassifier(), cv = 5, param_distributions = param_xgb,
                                  # scoring = 'precision', n_iter = 5, n_jobs = -1,random_state=42)
random_search.fit(X_train, y_train)
random_search.best_params_


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:00:57] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:00:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:00:59] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not 

{'estimator__subsample': 1.0,
 'estimator__scale_pos_weight': 5,
 'estimator__objective': 'binary:logistic',
 'estimator__nthread': -1,
 'estimator__n_estimators': 10,
 'estimator__min_child_weight': 1000,
 'estimator__max_depth': 3,
 'estimator__learning_rate': 0.5,
 'estimator__gamma': 1,
 'estimator__colsample_bytree': 0.6}

In [18]:
xgb_final_model = xgb.XGBClassifier(n_estimators = 10,learning_rate = 0.5,
                                    max_depth = 3,gamma = 1,subsample = 1.0,
                                    objective = 'binary:logistic',
                                    scale_pos_weight = 5,
                                    colsample_bytree = 0.6,
                                    min_child_weight = 10,
                                    nthread = -1)
xgb_final_model.fit(X_train,y_train)
y_train_pred = xgb_final_model.predict(X_train)
print(" Below is result on training data with pruning : ")
print(accuracy_score(y_train,y_train_pred))
print(confusion_matrix(y_train,y_train_pred))
print(classification_report(y_train,y_train_pred))


# Test data
y_pred = xgb_final_model.predict(X_test)
print(" Below is result on test data with pruning : ")
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

 Below is result on training data with pruning : 
0.8993323445970038
[[156907  36606]
 [  2355 191158]]
              precision    recall  f1-score   support

           0       0.99      0.81      0.89    193513
           1       0.84      0.99      0.91    193513

    accuracy                           0.90    387026
   macro avg       0.91      0.90      0.90    387026
weighted avg       0.91      0.90      0.90    387026

 Below is result on test data with pruning : 
0.8040626999360204
[[39925  9768]
 [   32   291]]
              precision    recall  f1-score   support

           0       1.00      0.80      0.89     49693
           1       0.03      0.90      0.06       323

    accuracy                           0.80     50016
   macro avg       0.51      0.85      0.47     50016
weighted avg       0.99      0.80      0.89     50016



In [32]:
from sklearn.metrics import roc_curve
import numpy as np

# Function to calculate the KS statistic
def calculate_ks_statistic(y_true, y_pred_proba):
    """
    y_true: True labels (0 or 1)
    y_pred_proba: Predicted probabilities for the positive class (class 1)
    """
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    ks_statistic = np.max(tpr - fpr)
    return ks_statistic



# For Test KS
ks_statistic_test = calculate_ks_statistic(y_test, y_pred)

# Print results
#print(f"KS Statistic on Training Data: {ks_statistic_train}")
print(f"KS Statistic on Test Data: {ks_statistic_test}")


KS Statistic on Test Data: 0.7043618716753685


In [ ]:
from sklearn.model_selection import GridSearchCV


grid_search = GridSearchCV(xgb.XGBClassifier(), cv = 5, param_grid = param_xgb,
                        scoring = 'f1', n_jobs = -1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(

KeyboardInterrupt



In [36]:
product_test=xgb_final_model.predict(product_test)

In [36]:
product_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62520 entries, 0 to 62519
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sku                62520 non-null  int64  
 1   national_inv       62520 non-null  int64  
 2   lead_time          62520 non-null  int64  
 3   in_transit_qty     62520 non-null  int64  
 4   forecast_3_month   62520 non-null  int64  
 5   forecast_6_month   62520 non-null  int64  
 6   forecast_9_month   62520 non-null  int64  
 7   sales_1_month      62520 non-null  int64  
 8   sales_3_month      62520 non-null  int64  
 9   sales_6_month      62520 non-null  int64  
 10  sales_9_month      62520 non-null  int64  
 11  min_bank           62520 non-null  int64  
 12  potential_issue    62520 non-null  int64  
 13  pieces_past_due    62520 non-null  int64  
 14  perf_6_month_avg   62520 non-null  float64
 15  perf_12_month_avg  62520 non-null  float64
 16  local_bo_qty       625